# building a credit scoring model

preparing a scorecard

In [37]:
import pandas as pd
import numpy as np
from sklearn import preprocessing
import matplotlib.pyplot as plt 
plt.rc("font", size=14)
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split
import seaborn as sns
sns.set_theme(style="white")
sns.set_theme(style="whitegrid", color_codes=True)

In [38]:
df_loan = pd.read_csv("./Data/loan.csv")
df_loan.head()

C:\Users\srika\AppData\Local\Temp\ipykernel_15696\186542136.py:1: DtypeWarning: Columns (19,55) have mixed types. Specify dtype option on import or set low_memory=False.
  df_loan = pd.read_csv("./Data/loan.csv")


,id,member_id,loan_amnt,funded_amnt,funded_amnt_inv,term,int_rate,installment,grade,sub_grade,...,total_bal_il,il_util,open_rv_12m,open_rv_24m,max_bal_bc,all_util,total_rev_hi_lim,inq_fi,total_cu_tl,inq_last_12m
0,1077501,1296599,5000.0,5000.0,4975.0,36 months,10.65,162.87,B,B2,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,1077430,1314167,2500.0,2500.0,2500.0,60 months,15.27,59.83,C,C4,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,1077175,1313524,2400.0,2400.0,2400.0,36 months,15.96,84.33,C,C5,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,1076863,1277178,10000.0,10000.0,10000.0,36 months,13.49,339.31,C,C1,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,1075358,1311748,3000.0,3000.0,3000.0,60 months,12.69,67.79,B,B5,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [39]:
#Analyzing target variable - loan_status
df_loan['loan_status'].value_counts()

loan_status
Current                                                601779
Fully Paid                                             207723
Charged Off                                             45248
Late (31-120 days)                                      11591
Issued                                                   8460
In Grace Period                                          6253
Late (16-30 days)                                        2357
Does not meet the credit policy. Status:Fully Paid       1988
Default                                                  1219
Does not meet the credit policy. Status:Charged Off       761
Name: count, dtype: int64

In [40]:
loan_filter = df_loan['loan_status'].isin(['Fully Paid','Charged Off','Default'])
df_loan = df_loan[loan_filter]
df_loan['loan_status'].value_counts()

loan_status
Fully Paid     207723
Charged Off     45248
Default          1219
Name: count, dtype: int64

In [41]:
def CreateTarget(status): 
    if status == 'Fully Paid':
        return 0
    else:
        return 1
    
df_loan['Late_Loan'] = df_loan['loan_status'].map(CreateTarget)
df_loan['Late_Loan'].value_counts()
df_loan['Late_Loan'].mean()

0.18280420158149416

In [42]:
features_missing_series = df_loan.isnull().sum() > len(df_loan)/10
features_missing_series = features_missing_series[features_missing_series == True]
features_missing_list =  features_missing_series.index.tolist()
df_loan = df_loan.drop(features_missing_list,axis =1)

In [43]:
df_loan_1 = df_loan.drop(['id','member_id','loan_status','url','zip_code','policy_code','application_type','last_pymnt_d','last_credit_pull_d','verification_status','pymnt_plan','funded_amnt','funded_amnt_inv','sub_grade','out_prncp','out_prncp_inv','total_pymnt_inv','total_pymnt','total_pymnt_inv','total_rec_prncp','total_rec_int','total_rec_late_fee','recoveries','collection_recovery_fee','last_pymnt_amnt','initial_list_status','earliest_cr_line'],axis =1)
df_loan_1['emp_length'].fillna('Unknown',inplace = True)
df_loan_1['emp_title'].fillna('Unknown',inplace = True)
df_loan_1['title'].fillna('Unknown',inplace = True)
df_loan_1['revol_util'].fillna(df_loan_1['revol_util'].mean(),inplace = True)
df_loan_1['collections_12_mths_ex_med'].fillna(df_loan_1['collections_12_mths_ex_med'].mean(),inplace = True)
df_loan_1.isnull().sum()

loan_amnt                     0
term                          0
int_rate                      0
installment                   0
grade                         0
emp_title                     0
emp_length                    0
home_ownership                0
annual_inc                    0
issue_d                       0
purpose                       0
title                         0
addr_state                    0
dti                           0
delinq_2yrs                   0
inq_last_6mths                0
open_acc                      0
pub_rec                       0
revol_bal                     0
revol_util                    0
total_acc                     0
collections_12_mths_ex_med    0
acc_now_delinq                0
Late_Loan                     0
dtype: int64

In [44]:
import pandas as pd
import numpy as np
import pandas.core.algorithms as algos
from pandas import Series
import scipy.stats as stats
import re
import traceback
import string

max_bin = 20
force_bin = 3

# Define a binning function
def mono_bin(Y, X, n=max_bin):
    df1 = pd.DataFrame({"X": X, "Y": Y})
    justmiss = df1[['X','Y']][df1.X.isnull()]
    notmiss = df1[['X','Y']][df1.X.notnull()]
    r = 0
    while np.abs(r) < 1:
        try:
            d1 = pd.DataFrame({"X": notmiss.X, "Y": notmiss.Y, "Bucket": pd.qcut(notmiss.X, n)})
            d2 = d1.groupby('Bucket', as_index=True)
            r, p = stats.spearmanr(d2.mean().X, d2.mean().Y)
            n = n - 1 
        except Exception as e:
            n = n - 1

    if len(d2) == 1:
        n = force_bin         
        bins = np.linspace(0, 1, n)
        quantiles = notmiss.X.quantile(bins)
        if len(np.unique(quantiles)) == 2:
            quantiles = np.insert(quantiles.values, 0, 1)
            quantiles[1] = quantiles[1] - (quantiles[1] / 2)
        d1 = pd.DataFrame({"X": notmiss.X, "Y": notmiss.Y, "Bucket": pd.cut(notmiss.X, np.unique(quantiles), include_lowest=True)}) 
        d2 = d1.groupby('Bucket', as_index=True, observed=False)

    
    d3 = pd.DataFrame({}, index=[])
    d3["MIN_VALUE"] = d2.min().X
    d3["MAX_VALUE"] = d2.max().X
    d3["COUNT"] = d2.count().Y
    d3["EVENT"] = d2.sum().Y
    d3["NONEVENT"] = d2.count().Y - d2.sum().Y
    d3 = d3.reset_index(drop=True)
    
    if len(justmiss.index) > 0:
        d4 = pd.DataFrame({'MIN_VALUE': np.nan}, index=[0])
        d4["MAX_VALUE"] = np.nan
        d4["COUNT"] = justmiss.count().Y
        d4["EVENT"] = justmiss.sum().Y
        d4["NONEVENT"] = justmiss.count().Y - justmiss.sum().Y
        d3 = d3.append(d4, ignore_index=True)
    
    d3["EVENT_RATE"] = d3.EVENT / d3.COUNT
    d3["NON_EVENT_RATE"] = d3.NONEVENT / d3.COUNT
    d3["DIST_EVENT"] = d3.EVENT / d3.sum().EVENT
    d3["DIST_NON_EVENT"] = d3.NONEVENT / d3.sum().NONEVENT
    d3["WOE"] = np.log(d3.DIST_EVENT / d3.DIST_NON_EVENT)
    d3["IV"] = (d3.DIST_EVENT - d3.DIST_NON_EVENT) * np.log(d3.DIST_EVENT / d3.DIST_NON_EVENT)
    d3["VAR_NAME"] = "VAR"
    d3 = d3[['VAR_NAME', 'MIN_VALUE', 'MAX_VALUE', 'COUNT', 'EVENT', 'EVENT_RATE', 'NONEVENT', 'NON_EVENT_RATE', 'DIST_EVENT', 'DIST_NON_EVENT', 'WOE', 'IV']]       
    d3 = d3.replace([np.inf, -np.inf], 0)
    d3.IV = d3.IV.sum()
    
    return d3

def char_bin(Y, X):
    df1 = pd.DataFrame({"X": X, "Y": Y})
    justmiss = df1[['X', 'Y']][df1.X.isnull()]
    notmiss = df1[['X', 'Y']][df1.X.notnull()]    
    df2 = notmiss.groupby('X', as_index=True)
    
    d3 = pd.DataFrame({}, index=[])
    d3["COUNT"] = df2.count().Y
    d3["MIN_VALUE"] = df2.sum().Y.index
    d3["MAX_VALUE"] = d3["MIN_VALUE"]
    d3["EVENT"] = df2.sum().Y
    d3["NONEVENT"] = df2.count().Y - df2.sum().Y
    
    if len(justmiss.index) > 0:
        d4 = pd.DataFrame({'MIN_VALUE': np.nan}, index=[0])
        d4["MAX_VALUE"] = np.nan
        d4["COUNT"] = justmiss.count().Y
        d4["EVENT"] = justmiss.sum().Y
        d4["NONEVENT"] = justmiss.count().Y - justmiss.sum().Y
        d3 = d3.append(d4, ignore_index=True)
    
    d3["EVENT_RATE"] = d3.EVENT / d3.COUNT
    d3["NON_EVENT_RATE"] = d3.NONEVENT / d3.COUNT
    d3["DIST_EVENT"] = d3.EVENT / d3.sum().EVENT
    d3["DIST_NON_EVENT"] = d3.NONEVENT / d3.sum().NONEVENT
    d3["WOE"] = np.log(d3.DIST_EVENT / d3.DIST_NON_EVENT)
    d3["IV"] = (d3.DIST_EVENT - d3.DIST_NON_EVENT) * np.log(d3.DIST_EVENT / d3.DIST_NON_EVENT)
    d3["VAR_NAME"] = "VAR"
    d3 = d3[['VAR_NAME', 'MIN_VALUE', 'MAX_VALUE', 'COUNT', 'EVENT', 'EVENT_RATE', 'NONEVENT', 'NON_EVENT_RATE', 'DIST_EVENT', 'DIST_NON_EVENT', 'WOE', 'IV']]      
    d3 = d3.replace([np.inf, -np.inf], 0)
    d3.IV = d3.IV.sum()
    d3 = d3.reset_index(drop=True)
    
    return d3

def data_vars(df1, target):
    iv_df = pd.DataFrame()  # Initialize iv_df as an empty DataFrame
    
    stack = traceback.extract_stack()
    filename, lineno, function_name, code = stack[-2]
    vars_name = re.compile(r'\((.*?)\).*$').search(code).groups()[0]
    final = (re.findall(r"[\w']+", vars_name))[-1]
    
    x = df1.dtypes.index
    count = -1
    
    for i in x:
        if i.upper() not in (final.upper()):
            if np.issubdtype(df1[i], np.number) and len(df1[i].unique()) > 2:
                conv = mono_bin(target, df1[i])
                conv["VAR_NAME"] = i
                count = count + 1
            else:
                conv = char_bin(target, df1[i])
                conv["VAR_NAME"] = i            
                count = count + 1
                
            if count == 0:
                iv_df = conv
            else:
                iv_df = pd.concat([iv_df, conv], ignore_index=True)  # Concatenate DataFrames
    
    iv = pd.DataFrame({'IV': iv_df.groupby('VAR_NAME').IV.max()})
    iv = iv.reset_index()
    return iv_df, iv

final_iv, IV = data_vars(df_loan_1, df_loan_1.Late_Loan)


C:\Users\srika\AppData\Local\Temp\ipykernel_15696\3784111533.py:22: FutureWarning: The default of observed=False is deprecated and will be changed to True in a future version of pandas. Pass observed=False to retain current behavior or observed=True to adopt the future default and silence this warning.
  d2 = d1.groupby('Bucket', as_index=True)
C:\Users\srika\AppData\Local\Temp\ipykernel_15696\3784111533.py:23: DeprecationWarning: Please import `spearmanr` from the `scipy.stats` namespace; the `scipy.stats.stats` namespace is deprecated and will be removed in SciPy 2.0.0.
  r, p = stats.spearmanr(d2.mean().X, d2.mean().Y)
C:\Users\srika\AppData\Local\Temp\ipykernel_15696\3784111533.py:22: FutureWarning: The default of observed=False is deprecated and will be changed to True in a future version of pandas. Pass observed=False to retain current behavior or observed=True to adopt the future default and silence this warning.
  d2 = d1.groupby('Bucket', as_index=True)
C:\Users\srika\AppData\

In [45]:
final_iv

,VAR_NAME,MIN_VALUE,MAX_VALUE,COUNT,EVENT,EVENT_RATE,NONEVENT,NON_EVENT_RATE,DIST_EVENT,DIST_NON_EVENT,WOE,IV
0,loan_amnt,500.0,6000.0,49948,7691,0.153980,42257,0.846020,0.165515,0.203430,-0.206256,0.025570
1,loan_amnt,6025.0,9000.0,36665,5967,0.162744,30698,0.837256,0.128414,0.147783,-0.140490,0.025570
2,loan_amnt,9025.0,12000.0,51177,8940,0.174688,42237,0.825312,0.192395,0.203333,-0.055298,0.025570
3,loan_amnt,12025.0,15600.0,31879,5800,0.181938,26079,0.818062,0.124820,0.125547,-0.005809,0.025570
4,loan_amnt,15625.0,21000.0,42829,8786,0.205141,34043,0.794859,0.189080,0.163887,0.142998,0.025570
...,...,...,...,...,...,...,...,...,...,...,...,...
181457,total_acc,44.0,150.0,18917,3054,0.161442,15863,0.838558,0.065724,0.076366,-0.150074,0.007369
181458,collections_12_mths_ex_med,0.0,1.0,254081,46448,0.182808,207633,0.817192,0.999591,0.999567,0.000024,0.000001
181459,collections_12_mths_ex_med,2.0,6.0,109,19,0.174312,90,0.825688,0.000409,0.000433,-0.057908,0.000001
181460,acc_now_delinq,0.0,1.0,254148,46455,0.182787,207693,0.817213,0.999742,0.999856,-0.000114,0.000066


In [46]:
IV.sort_values('IV')

,VAR_NAME,IV
3,collections_12_mths_ex_med,0.000001
0,acc_now_delinq,0.000066
16,pub_rec,0.000132
15,open_acc,0.001109
4,delinq_2yrs,0.001647
18,revol_bal,0.001897
22,total_acc,0.007369
6,emp_length,0.011091
10,inq_last_6mths,0.014768
1,addr_state,0.015219


In [50]:
IV.to_csv(r'.\Data\test.csv')

In [51]:
transform_vars_list = df_loan_1.columns.difference(['Late_Loan'])
transform_prefix = 'new_' # leave this value blank if you need replace the original column values

In [52]:
transform_vars_list
df=df_loan_1
df.head()

,loan_amnt,term,int_rate,installment,grade,emp_title,emp_length,home_ownership,annual_inc,issue_d,...,delinq_2yrs,inq_last_6mths,open_acc,pub_rec,revol_bal,revol_util,total_acc,collections_12_mths_ex_med,acc_now_delinq,Late_Loan
0,5000.0,36 months,10.65,162.87,B,Unknown,10+ years,RENT,24000.0,Dec-2011,...,0.0,1.0,3.0,0.0,13648.0,83.7,9.0,0.0,0.0,0
1,2500.0,60 months,15.27,59.83,C,Ryder,< 1 year,RENT,30000.0,Dec-2011,...,0.0,5.0,3.0,0.0,1687.0,9.4,4.0,0.0,0.0,1
2,2400.0,36 months,15.96,84.33,C,Unknown,10+ years,RENT,12252.0,Dec-2011,...,0.0,2.0,2.0,0.0,2956.0,98.5,10.0,0.0,0.0,0
3,10000.0,36 months,13.49,339.31,C,AIR RESOURCES BOARD,10+ years,RENT,49200.0,Dec-2011,...,0.0,1.0,10.0,0.0,5598.0,21.0,37.0,0.0,0.0,0
5,5000.0,36 months,7.90,156.46,A,Veolia Transportaton,3 years,RENT,36000.0,Dec-2011,...,0.0,3.0,9.0,0.0,7963.0,28.3,12.0,0.0,0.0,0


In [53]:
for var in transform_vars_list:
    small_df = final_iv[final_iv['VAR_NAME'] == var]
    transform_dict = dict(zip(small_df.MAX_VALUE,small_df.WOE))
    replace_cmd = ''
    replace_cmd1 = ''
    for i in sorted(transform_dict.items()):
        replace_cmd = replace_cmd + str(i[1]) + str(' if x <= ') + str(i[0]) + ' else '
        replace_cmd1 = replace_cmd1 + str(i[1]) + str(' if x == "') + str(i[0]) + '" else '
    replace_cmd = replace_cmd + '0'
    replace_cmd1 = replace_cmd1 + '0'
    if replace_cmd != '0':
        try:
            df[transform_prefix + var] = df[var].apply(lambda x: eval(replace_cmd))
        except:
            df[transform_prefix + var] = df[var].apply(lambda x: eval(replace_cmd1))

MemoryError: 

In [ ]:
df['new_grade'].value_counts()

In [ ]:
df['grade'].value_counts()

In [ ]:
df_loan.groupby('grade').mean()

In [ ]:
y=df['Late_Loan']
y.mean()
features=[#'new_acc_now_delinq',
'new_addr_state',
'new_annual_inc',
#'new_collections_12_mths_ex_med',
'new_delinq_2yrs',
'new_dti',
'new_emp_length',
'new_grade',
'new_home_ownership',
'new_inq_last_6mths',
#'new_installment',
'new_int_rate',
'new_issue_d',
'new_loan_amnt',
'new_open_acc',
#'new_pub_rec',
'new_purpose',
#'new_revol_bal',
'new_revol_util',
'new_term'
]
X=df[features]


In [55]:
import statsmodels.api as sm
logit_model=sm.Logit(y,X)
result=logit_model.fit()
print(result.summary2())

NameError: name 'y' is not defined

In [ ]:
from sklearn.linear_model import LogisticRegression
from sklearn import tree
from sklearn.tree import DecisionTreeClassifier
from sklearn import metrics

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=0)
logreg = LogisticRegression()
logreg.fit(X_train, y_train)

dtree = DecisionTreeClassifier(criterion = "gini", random_state = 100,max_depth=5, min_samples_leaf=5)
dtree.fit(X_train, y_train)

In [56]:

y_pred = logreg.predict(X)
y_pred_tree=dtree.predict(X_test)
print('Accuracy of logistic regression classifier on test set: {:.2f}'.format(logreg.score(X_test, y_test)))

NameError: name 'logreg' is not defined

In [57]:
from sklearn.metrics import confusion_matrix
confusion_matrix = confusion_matrix(y, y_pred)
#print(confusion_matrix)
pd.crosstab(y,y_pred)

NameError: name 'y' is not defined

In [58]:
from sklearn.metrics import classification_report
print(classification_report(y, y_pred))

NameError: name 'y' is not defined

In [59]:
from sklearn.metrics import roc_auc_score
from sklearn.metrics import roc_curve
logit_roc_auc = roc_auc_score(y_test, logreg.predict(X_test))
tree_roc_auc=roc_auc_score(y_test, dtree.predict(X_test))

fpr, tpr, thresholds = roc_curve(y_test, logreg.predict_proba(X_test)[:,1])
fpr, tpr, thresholds = roc_curve(y_test, dtree.predict_proba(X_test)[:,1])
plt.figure()
plt.plot(fpr, tpr, label='Logistic Regression (area = %0.2f)' % logit_roc_auc)
plt.plot(fpr, tpr, label='Tree Regression (area = %0.2f)' % tree_roc_auc)
plt.plot([0, 1], [0, 1],'r--')
plt.xlim([0.0, 1.0])
plt.ylim([0.0, 1.05])
plt.xlabel('False Positive Rate')
plt.ylabel('True Positive Rate')
plt.title('Receiver operating characteristic')
plt.legend(loc="lower right")
plt.savefig('Log_ROC')
plt.show()

NameError: name 'y_test' is not defined